## Paso 1: Instalar las dependencias y la configuración del notebook en GPU

In [1]:
!pip install -U "tensorflow==2.20.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 85.2 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboard-2.19.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.19.0
    Uninstalling tensorflow-2.19.0:
      Successfully uninstalled tensorflow-2.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.20.0 which is incompatible.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.20.0 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.20.0 which is inc

## Paso 2: Importar las librerías necesarias

In [2]:
import tensorflow as tf
# DataSet de valoracion de peliculas
from tensorflow.keras.datasets import imdb # pyright: ignore[reportMissingImports]

In [3]:
tf.__version__

'2.20.0'

## Paso 3: Pre procesado de datos


### Configurar parámetros del dataset

In [4]:
# Vamos a definir el numero maximo de palabras
number_of_words = 20000
# Y valoraciones de tamaño 100
max_len = 100
# Con esto logramos la uniformidad en la longitud

### Carga del dataset de IMDB

In [5]:
# Al cargar el dataset le podemos pasar por parametro el numero maximo de palabras, entonces nos quedaremos con las 20.000 palabras mas frecuente
# El proceso de carla lo divide en entrenamiento y testing
# X para las valoraciones e Y para para la clasificaicion (positiva o negativa)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=number_of_words)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


### Cortar secuencias de texto de la misma longitud (100 elementos)

In [6]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_len)

In [7]:
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_len)

### Configurar parámetros de la capa de Embedding

In [8]:
# El numero del vocabulario sera de 20.000
vocab_size = number_of_words
vocab_size

20000

In [9]:
# Definiremos el tamaño de inserción para las columnas
embed_size = 128

## Paso 4: Construir la Red Neuronal Recurrente

### Definir el modelo
* Primero definiremos el modelo como Secuencial
* Le añadiremos todo el conjunto de entrenamiento correspondiente a las valoraciones

In [10]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(X_train.shape[1],)))

### Añadir la capa de embedding
* En la capa de Embedding le suministramos los textos, antes de pasarselo a la red neuronal
* Crearemos un vector de palabras para transformalo a un formato numerico
* Como primer parametro del Embedding le pasamos la dimencion de entrada (numero de palabras a analizar)
* Segundo parametro es la dimencion de salida, 128 neuronas de la capa oculta



In [11]:
model.add(tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_size))

### Añadir la capa de LSTM

- unidades: 128
- función de activación: tanh (Tangente hiperbolica)

In [12]:
model.add(tf.keras.layers.LSTM(units=128, activation='tanh'))

### Añadir la capa totalmente conectada de salida

- unidades: 1
- función de activación: sigmoid (Lo transforma a probabilidades)
* Cercano a 1 es una valoracion positiva y cercano a 0 sera negativa

In [13]:
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Compilar el modelo
* Cambiaremos el optimizador de Adam por rmsprop que es especialista para Redes neuronales recurrente
* Como es positivo o negativo la funcion de perdida sera binaria
* Y la metrica sera accurasy (aciertas o fallas)

In [14]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 128)       │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,691,713 (10.27 MB)

 Trainable params: 2,691,713 (10.27 MB)

 Non-trainable params: 0 (0.00 B)

### Entrenar el modelo
* X_train: Todas las fraces para entrenar
* y_traint: Valoracion positiva o negativa
* Pasaremos 3 veces
* Las fraces se las pasaremos en bloques de 128

In [16]:
model.fit(X_train, y_train, epochs=3, batch_size=128)

Epoch 1/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.5984 - loss: 0.6511
Epoch 2/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8190 - loss: 0.4034
Epoch 3/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8638 - loss: 0.3247


### Evaluar el modelo

In [17]:
test_loss, test_acurracy = model.evaluate(X_test, y_test)

782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8542 - loss: 0.3389


In [18]:
print("Test accuracy: {}".format(test_acurracy))

Test accuracy: 0.8546000123023987
